# Object Detection using TAO DSSD

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 DSSD model on the KITTI dataset
* Prune the trained DSSD model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model

## Table of Contents

This notebook shows an example usecase of DSSD object detection using Train Adapt Optimize (TAO) Toolkit.

1. [Set up env variables and map drives](#head-0)
2. [Prepare dataset and pre-trained model](#head-2) <br>
    2.1 [Download pre-trained model](#head-2-1) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Prune trained models](#head-6)
7. [Retrain pruned models](#head-7)
8. [Evaluate retrained model](#head-8)
9. [Visualize inferences](#head-9)
10. [Model Export](#head-10)
11. [Verify deployed model](#head-11)

## 1. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/dssd`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.



In [20]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variable with your key.")
%env KEY=nvidia_tlt
%env GPU_INDEX=0

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/dssd
%env LOCAL_PROJECT_DIR=/home/jupyter/imported_files/files
os.environ["LOCAL_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data")
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "dssd")

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "dssd/specs"
)

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

Please replace the variable with your key.
env: KEY=nvidia_tlt
env: GPU_INDEX=0
env: LOCAL_PROJECT_DIR=/home/jupyter
total 20
-rw-r--r-- 1 jupyter jupyter 1700 Jun 15 19:23 dssd_train_resnet18_kitti_seq.txt
-rw-r--r-- 1 jupyter jupyter  608 Sep 12 18:13 dssd_tfrecords_kitti_train.txt
-rw-r--r-- 1 jupyter jupyter 1580 Sep 12 18:28 dssd_train_resnet18_kitti.txt
-rw-r--r-- 1 jupyter jupyter 1566 Sep 12 21:52 dssd_retrain_resnet18_kitti.txt
-rw-r--r-- 1 jupyter jupyter 1634 Sep 13 17:18 dssd_retrain_resnet18_kitti_seq.txt


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.

In [6]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

In [7]:
# Check the dataset is present
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

Found Image zip file.
Found Labels zip file.


In [8]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/data_object_image_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_label_2.zip -d $LOCAL_DATA_DIR

Archive:  /home/jupyter/data/data_object_image_2.zip
   creating: /home/jupyter/data/training/image_2/
 extracting: /home/jupyter/data/training/image_2/002480.png  
 extracting: /home/jupyter/data/training/image_2/005952.png  
 extracting: /home/jupyter/data/training/image_2/000709.png  
 extracting: /home/jupyter/data/training/image_2/000814.png  
 extracting: /home/jupyter/data/training/image_2/006192.png  
 extracting: /home/jupyter/data/training/image_2/006017.png  
 extracting: /home/jupyter/data/training/image_2/002731.png  
 extracting: /home/jupyter/data/training/image_2/005295.png  
 extracting: /home/jupyter/data/training/image_2/005347.png  
 extracting: /home/jupyter/data/training/image_2/005326.png  
 extracting: /home/jupyter/data/training/image_2/005713.png  
 extracting: /home/jupyter/data/training/image_2/007000.png  
 extracting: /home/jupyter/data/training/image_2/004353.png  
 extracting: /home/jupyter/data/training/image_2/003969.png  
 extracting: /home/jupyter/da

In [2]:
# verify
!ls -l $LOCAL_DATA_DIR/

total 12280852
-rw-r--r-- 1 jupyter jupyter 12569945557 May 11  2018  data_object_image_2.zip
-rw-r--r-- 1 jupyter jupyter     5601213 Sep 12 17:56  data_object_label_2.zip
-rw-r--r-- 1 jupyter jupyter        2556 Sep 12 17:53 'index.html?url=https:%2F%2Fs3.eu-central-1.amazonaws.com%2Favg-kitti%2Fdata_object_label_2.zip'
-rw-r--r-- 1 jupyter jupyter        2556 Sep 12 17:53 'index.html?url=https:%2F%2Fs3.eu-central-1.amazonaws.com%2Favg-kitti%2Fdata_object_label_2.zip.1'
drwxr-xr-x 2 jupyter jupyter        4096 Sep 12 23:22  test_samples
drwxr-xr-x 3 jupyter jupyter        4096 Sep 12 18:08  testing
drwxr-xr-x 2 jupyter jupyter        4096 Sep 12 18:21  tfrecords
drwxr-xr-x 4 jupyter jupyter        4096 Sep 12 18:09  training
drwxr-xr-x 4 jupyter jupyter        4096 Sep 12 18:12  val
-rw-r--r-- 1 jupyter jupyter         969 Sep 12 17:53  wget-log
-rw-r--r-- 1 jupyter jupyter         973 Sep 12 17:53  wget-log.1


In [3]:
# Generate val dataset out of training dataset
!python3 ../ssd/generate_val_dataset.py --input_image_dir=$LOCAL_DATA_DIR/training/image_2 \
                                        --input_label_dir=$LOCAL_DATA_DIR/training/label_2 \
                                        --output_dir=$LOCAL_DATA_DIR/val

This script will not run as output image path already exists.


In [12]:
print("TFRecords conversion spec file:")
!cat $LOCAL_SPECS_DIR/dssd_tfrecords_kitti_train.txt

TFRecords conversion spec file:
kitti_config {
  root_directory_path: "/home/jupyter/data/training"
  image_dir_name: "image_2"
  label_dir_name: "label_2"
  image_extension: ".png"
  partition_mode: "random"
  num_partitions: 2
  val_split: 0
  num_shards: 10
}
image_directory_path: "/home/jupyter/data/training"
target_class_mapping {
    key: "car"
    value: "car"
}
target_class_mapping {
    key: "pedestrian"
    value: "pedestrian"
}
target_class_mapping {
    key: "cyclist"
    value: "cyclist"
}
target_class_mapping {
    key: "van"
    value: "car"
}
target_class_mapping {
    key: "person_sitting"
    value: "pedestrian"
}


In [13]:
# Creating a new directory for the output tfrecords dump.
print("Converting the training set to TFRecords.")
!mkdir -p $LOCAL_DATA_DIR/tfrecords && rm -rf $LOCAL_DATA_DIR/tfrecords/*
!dssd dataset_convert \
          -d $LOCAL_SPECS_DIR/dssd_tfrecords_kitti_train.txt \
          -o $LOCAL_DATA_DIR/tfrecords/kitti_train

Converting the training set to TFRecords.
Using TensorFlow backend.
Using TensorFlow backend.
2022-09-12 18:14:40,681 [INFO] iva.detectnet_v2.dataio.build_converter: Instantiating a kitti converter
2022-09-12 18:14:40,681 [INFO] root: Instantiating a kitti converter
2022-09-12 18:14:40,681 [INFO] root: Generating partitions
2022-09-12 18:14:40,703 [INFO] iva.detectnet_v2.dataio.kitti_converter_lib: Num images in
Train: 6733	Val: 0
2022-09-12 18:14:40,703 [INFO] root: Num images in
Train: 6733	Val: 0
2022-09-12 18:14:40,704 [INFO] iva.detectnet_v2.dataio.kitti_converter_lib: Skipped validation data...
2022-09-12 18:14:40,704 [INFO] root: Skipped validation data.
2022-09-12 18:14:40,710 [INFO] iva.detectnet_v2.dataio.dataset_converter_lib: Writing partition 0, shard 0
2022-09-12 18:14:40,710 [INFO] root: Writing partition 0, shard 0

2022-09-12 18:14:40,710 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/

In [14]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/

total 5469228
-rw-r--r-- 1 jupyter jupyter 562230269 Sep 12 18:15 kitti_train-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 jupyter jupyter 564127829 Sep 12 18:16 kitti_train-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 jupyter jupyter 559431033 Sep 12 18:16 kitti_train-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 jupyter jupyter 556836758 Sep 12 18:17 kitti_train-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 jupyter jupyter 560464942 Sep 12 18:18 kitti_train-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 jupyter jupyter 558708543 Sep 12 18:18 kitti_train-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 jupyter jupyter 559013750 Sep 12 18:19 kitti_train-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 jupyter jupyter 559448685 Sep 12 18:20 kitti_train-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 jupyter jupyter 560573069 Sep 12 18:20 kitti_train-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 jupyter jupyter 559468890 Sep 12 18:21 kitti_train-fold-000-of-0

Additionally, if you have your own dataset already in a volume (or folder), you can mount the volume on `LOCAL_DATA_DIR` (or create a soft link). Below shows an example:
```bash
# if your dataset is in /dev/sdc1
mount /dev/sdc1 $LOCAL_DATA_DIR

# if your dataset is in folder /var/dataset
ln -sf /var/dataset $LOCAL_DATA_DIR
```

### 2.1 Download pre-trained model <a class="anchor" id="head-2-1"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [21]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2022-09-13 17:56:09--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.32.164.11, 13.32.164.19, 13.32.164.13, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.32.164.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34018893 (32M) [application/zip]
Saving to: ‘/home/jupyter/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  32.44M  49.7MB/s    in 0.7s    

2022-09-13 17:56:10 (49.7 MB/s) - ‘/home/jupyter/ngccli/ngccli_cat_linux.zip’ saved [34018893/34018893]

Archive:  /home/jupyter/ngccli/ngccli_cat_linux.zip
   creating: /home/jupyter/ngccli/ngc-cli/
   creating: /home/jupyter/ngccli/ngc-cli/yarl/
  inflating: /home/jupyter/ngccli/ngc-cli/yarl/_quoting_c.cpython-39-x86_64-linux-gnu.so  
  inflating: /home/jupyter/ngccli/ngc-cli/libselinux.so.1  
   creating: /home/jupyter/ngccli/ngc-cli/prettytable-2.0.0.dist-info/
  inflating: /home/jupyter/

In [5]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| vgg19 | 77.56 | 80    | 1     | V100  | 153.7 | 153.7 | UPLOA | Aug   |
|       |       |       |       |       |       | 2 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| vgg16 | 77.17 | 80    | 1     | V100  | 113.2 | 113.1 | UPLOA | Aug   |
|       |       |       |       |       |       | 6 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| squee | 65.13 | 80    | 1     | V100  | 6.5   | 6.46  | UPLOA | Aug   |
| zenet |       |       |       |     

In [17]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [18]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18 --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

Downloaded 82.38 MB in 6s, Download speed: 13.71 MB/s               
--------------------------------------------------------------------------------
   Transfer id: pretrained_object_detection_vresnet18
   Download status: Completed
   Downloaded local path: /home/jupyter/dssd/pretrained_resnet18/pretrained_object_detection_vresnet18
   Total files downloaded: 1
   Total downloaded size: 82.38 MB
   Started at: 2022-09-12 18:23:51.561207
   Completed at: 2022-09-12 18:23:57.576861
   Duration taken: 6s
--------------------------------------------------------------------------------


In [19]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18

Check that model is downloaded into dir.
total 91096
-rw------- 1 jupyter jupyter 93278448 Sep 12 18:23 resnet_18.hdf5


## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Dataset for the train datasets
    * In order to use the newly generated dataset, update the dataset_config parameter in the spec file at `$LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt` 
* Augmentation parameters for on the fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* Whether to use quantization aware training (QAT)

In [ ]:
# To enable QAT training on sample spec file, uncomment following lines
# !sed -i "s/enable_qat: false/enable_qat: true/g" $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt
# !sed -i "s/enable_qat: false/enable_qat: true/g" $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt

In [ ]:
# By default, the sample spec file disables QAT training. You can force non-QAT training by running lines below
# !sed -i "s/enable_qat: true/enable_qat: false/g" $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt
# !sed -i "s/enable_qat: true/enable_qat: false/g" $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt

In [20]:
!cat $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt

random_seed: 42
dssd_config {
  aspect_ratios_global: "[1.0, 2.0, 0.5, 3.0, 1.0/3.0]"
  scales: "[0.05, 0.1, 0.25, 0.4, 0.55, 0.7, 0.85]"
  two_boxes_for_ar1: true
  clip_boxes: false
  variances: "[0.1, 0.1, 0.2, 0.2]"
  arch: "resnet"
  nlayers: 18
  pred_num_channels: 512
  freeze_bn: false
  freeze_blocks: 0
}
training_config {
  batch_size_per_gpu: 10
  num_epochs: 80
  enable_qat: false
  learning_rate {
  soft_start_annealing_schedule {
    min_learning_rate: 5e-5
    max_learning_rate: 9e-3
    soft_start: 0.1
    annealing: 0.8
    }
  }
  regularizer {
    type: L1
    weight: 3e-5
  }
}
eval_config {
  validation_period_during_training: 10
  average_precision_mode: SAMPLE
  batch_size: 16
  matching_iou_threshold: 0.5
}
nms_config {
  confidence_threshold: 0.01
  clustering_iou_threshold: 0.6
  top_k: 200
}
augmentation_config {
    output_width: 300
    output_height: 300
    output_channel: 3
}
dataset_config {
  data_sources: {
    tfrecords_path: "/home/jupyter/data/tfre

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [21]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

In [22]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!dssd train --gpus 1 --gpu_index=$GPU_INDEX \
                -e $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt \
                -r $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned \
                -k $KEY \
                -m $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5

To run with multigpu, please change --gpus based on the number of available GPUs in your machine.
Using TensorFlow backend.
Using TensorFlow backend.

2022-09-12 18:30:01,699 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/ssd/scripts/train.py:64: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2022-09-12 18:30:01,699 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/ssd/scripts/train.py:67: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2022-09-12 18:30:26,041 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_train_resnet18_kitti.txt

2022-09-12 18:30:26,047 [WARNING] tensorflow: From /usr/local/lib/p

In [ ]:
print("To resume from checkpoint, please uncomment and run this instead. Change last two arguments accordingly.")
# !dssd train --gpus 1 --gpu_index=$GPU_INDEX \
#                 -e $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt \
#                 -r $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned \
#                 -k $KEY \
#                 -m $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights/dssd_resnet18_epoch_001.tlt \
#                 --initial_epoch 2

In [6]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights

Model for each epoch:
---------------------
total 13G
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:36 dssd_resnet18_epoch_001.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:37 dssd_resnet18_epoch_002.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:39 dssd_resnet18_epoch_003.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:41 dssd_resnet18_epoch_004.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:42 dssd_resnet18_epoch_005.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:44 dssd_resnet18_epoch_006.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:46 dssd_resnet18_epoch_007.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:47 dssd_resnet18_epoch_008.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:49 dssd_resnet18_epoch_009.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:50 dssd_resnet18_epoch_010.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:53 dssd_resnet18_epoch_011.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:54 dssd_resnet18_epoch_012.tlt
-rw-r--r-- 1 jupyter jupyter 157M Sep 12 18:56

In [24]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/dssd_training_log_resnet18.csv
%set_env EPOCH=080

epoch,AP_car,AP_cyclist,AP_pedestrian,loss,lr,mAP,validation_loss
1,nan,nan,nan,21.010191,9.5692754e-05,nan,nan
2,nan,nan,nan,18.364367,0.00018314207,nan,nan
3,nan,nan,nan,17.923998,0.00035050738,nan,nan
4,nan,nan,nan,17.37588,0.00067082036,nan,nan
5,nan,nan,nan,16.879118,0.001283853,nan,nan
6,nan,nan,nan,16.215155,0.0024571086,nan,nan
7,nan,nan,nan,15.5776,0.00470255,nan,nan
8,nan,nan,nan,14.873452,0.008999999,nan,nan
9,nan,nan,nan,13.965604,0.009,nan,nan
10,0.7478859998704319,0.0303030303030303,0.22912890301292793,12.88441,0.009,0.3357726443954634,50.627040455048096
11,nan,nan,nan,12.019656,0.009,nan,nan
12,nan,nan,nan,11.254775,0.009,nan,nan
13,nan,nan,nan,10.628046,0.009,nan,nan
14,nan,nan,nan,10.047169,0.009,nan,nan
15,nan,nan,nan,9.465202,0.009,nan,nan
16,nan,nan,nan,8.990822,0.009,nan,nan
17,nan,nan,nan,8.627666,0.009,nan,nan
18,nan,nan,nan,8.236545,0.009,nan,nan
19,nan,nan,nan,7.968424,0.009,nan,nan
20,0.8029276824892474,0.11073995955715571,0.245706632223205,7.6540756,0.009,0.3

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [25]:
!dssd evaluate --gpu_index=$GPU_INDEX \
                   -e $LOCAL_SPECS_DIR/dssd_train_resnet18_kitti.txt \
                   -m $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights/dssd_resnet18_epoch_$EPOCH.tlt \
                   -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-12 20:55:11,108 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_train_resnet18_kitti.txt

2022-09-12 20:55:11,125 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2022-09-12 20:55:11,126 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2022-09-12 20:55:11,128 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2022-09-12 20:55:13,777 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/

## 6. Prune trained models <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* A key to save and load the model
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.5` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [26]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [27]:
!dssd prune --gpu_index=$GPU_INDEX \
                -m $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights/dssd_resnet18_epoch_$EPOCH.tlt \
                -o $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/dssd_resnet18_pruned.tlt \
                -eq intersection \
                -pth 0.1 \
                -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-12 21:14:59,402 [INFO] modulus.pruning.pruning: Exploring graph for retainable indices
2022-09-12 21:15:08,452 [INFO] modulus.pruning.pruning: Pruning model and appending pruned nodes to new graph
2022-09-12 21:18:41,519 [INFO] iva.common.magnet_prune: Pruning ratio (pruned model / original model): 0.33617615170624016
2022-09-12 21:18:42,657 [INFO] root: Pruning ratio (pruned model / original model): 0.33617615170624016
2022-09-12 21:18:42,658 [INFO] root: {
    "pruning_ratio": 0.33617615170624016,
    "size": 26.87982177734375,
    "param_count": 6.85624
}


In [7]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

total 27528
-rw-r--r-- 1 jupyter jupyter 28185536 Sep 12 21:18 dssd_resnet18_pruned.tlt


## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification
* WARNING: training will take several hours or one day to complete

In [30]:
# Printing the retrain spec file. 
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!cat $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt

random_seed: 42
dssd_config {
  aspect_ratios_global: "[1.0, 2.0, 0.5, 3.0, 1.0/3.0]"
  scales: "[0.05, 0.1, 0.25, 0.4, 0.55, 0.7, 0.85]"
  two_boxes_for_ar1: true
  clip_boxes: false
  variances: "[0.1, 0.1, 0.2, 0.2]"
  arch: "resnet"
  nlayers: 18
  pred_num_channels: 512
  freeze_bn: false
}
training_config {
  batch_size_per_gpu: 24
  num_epochs: 20
  enable_qat: false
  learning_rate {
  soft_start_annealing_schedule {
    min_learning_rate: 5e-5
    max_learning_rate: 1e-2
    soft_start: 0.1
    annealing: 0.5
    }
  }
  regularizer {
    type: NO_REG
    weight: 3e-09
  }
}
eval_config {
  validation_period_during_training: 10
  average_precision_mode: SAMPLE
  batch_size: 16
  matching_iou_threshold: 0.5
}
nms_config {
  confidence_threshold: 0.01
  clustering_iou_threshold: 0.6
  top_k: 200
}
augmentation_config {
    output_width: 300
    output_height: 300
    output_channel: 3
}
dataset_config {
  data_sources: {
    tfrecords_path: "/home/jupyter/data/tfrecords/kitti_tr

In [31]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain

In [32]:
# Retraining using the pruned model as pretrained weights 
!dssd train --gpus 1 --gpu_index=$GPU_INDEX \
                -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
                -r $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain \
                -m $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/dssd_resnet18_pruned.tlt \
                -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.

2022-09-12 21:53:00,660 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/ssd/scripts/train.py:64: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


2022-09-12 21:53:00,661 [WARNING] tensorflow: From /root/.cache/bazel/_bazel_root/ed34e6d125608f91724fda23656f1726/execroot/ai_infra/bazel-out/k8-fastbuild/bin/magnet/packages/iva/build_wheel.runfiles/ai_infra/iva/ssd/scripts/train.py:67: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2022-09-12 21:53:01,201 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_retrain_resnet18_kitti.txt

2022-09-12 21:53:01,206 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is

In [8]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

total 1085840
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:28 dssd_resnet18_epoch_001.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:30 dssd_resnet18_epoch_002.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:31 dssd_resnet18_epoch_003.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:32 dssd_resnet18_epoch_004.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:33 dssd_resnet18_epoch_005.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:35 dssd_resnet18_epoch_006.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:36 dssd_resnet18_epoch_007.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:37 dssd_resnet18_epoch_008.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:38 dssd_resnet18_epoch_009.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:40 dssd_resnet18_epoch_010.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:41 dssd_resnet18_epoch_011.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 Sep 12 22:43 dssd_resnet18_epoch_012.tlt
-rw-r--r-- 1 jupyter jupyter 55594616 

In [9]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/dssd_training_log_resnet18.csv
%set_env EPOCH=020

epoch,AP_car,AP_cyclist,AP_pedestrian,loss,lr,mAP,validation_loss
1,nan,nan,nan,2.7830064,0.0007071067,nan,nan
2,nan,nan,nan,2.460103,0.009999999,nan,nan
3,nan,nan,nan,2.6592753,0.01,nan,nan
4,nan,nan,nan,2.6416962,0.01,nan,nan
5,nan,nan,nan,2.6232266,0.01,nan,nan
6,nan,nan,nan,2.6095343,0.01,nan,nan
7,nan,nan,nan,2.6030345,0.01,nan,nan
8,nan,nan,nan,2.5502665,0.01,nan,nan
9,nan,nan,nan,2.5651093,0.01,nan,nan
10,0.8730776742305056,0.47411354933951305,0.4855056263054613,2.5584562,0.01,0.6108989499584934,33.19628704295439
11,nan,nan,nan,2.4822495,0.00588704,nan,nan
12,nan,nan,nan,2.3976922,0.0034657242,nan,nan
13,nan,nan,nan,2.3381338,0.0020402856,nan,nan
14,nan,nan,nan,2.3025405,0.0012011244,nan,nan
15,nan,nan,nan,2.286468,0.0007071068,nan,nan
16,nan,nan,nan,2.269099,0.0004162766,nan,nan
17,nan,nan,nan,2.2692869,0.0002450637,nan,nan
18,nan,nan,nan,2.2348058,0.00014426999,nan,nan
19,nan,nan,nan,2.2585645,8.493232e-05,nan,nan
20,0.8817164663513181,0.5486997045562476,0.5366320297582091,2.2

## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [2]:
!dssd evaluate --gpu_index=$GPU_INDEX \
                   -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
                   -m $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/dssd_resnet18_epoch_020.tlt \
                   -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-12 23:10:09,699 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_retrain_resnet18_kitti.txt

2022-09-12 23:10:09,702 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2022-09-12 23:10:09,702 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2022-09-12 23:10:09,705 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2022-09-12 23:10:10,556 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backen

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `infer` tool to generate inferences on the trained models and visualize the results.

In [3]:
# Copy some test images
!mkdir -p $LOCAL_DATA_DIR/test_samples
!cp $LOCAL_DATA_DIR/testing/image_2/00000* $LOCAL_DATA_DIR/test_samples

In [ ]:
# Running inference for detection on n images
!dssd inference --gpu_index=$GPU_INDEX -i $LOCAL_DATA_DIR/test_samples \
                    -o $LOCAL_EXPERIMENT_DIR/dssd_infer_images \
                    -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
                    -m $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/dssd_resnet18_epoch_020.tlt \
                   -l $LOCAL_EXPERIMENT_DIR/dssd_infer_labels \
                   -k $KEY

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-12 23:37:40,818 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_retrain_resnet18_kitti.txt

2022-09-12 23:37:40,822 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2022-09-12 23:37:40,822 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2022-09-12 23:37:40,826 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


2022-09-12 23:37:41,658 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backen

The `tao` inference tool produces two outputs. 
1. Overlain images in `$USER_EXPERIMENT_DIR/dssd_infer_images`
2. Frame by frame bbox labels in kitti format located in `$USER_EXPERIMENT_DIR/dssd_infer_labels`

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'dssd_infer_images' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export <a class="anchor" id="head-10"></a>

If you trained a non-QAT model, you may export in FP32, FP16 or INT8 mode using the code block below. For INT8, you need to provide calibration image directory.

In [10]:
# tao <task> export will fail if .etlt already exists. So we clear the export folder before tao <task> export
!rm -rf $LOCAL_EXPERIMENT_DIR/export
!mkdir -p $LOCAL_EXPERIMENT_DIR/export
# Export in FP32 mode. Change --data_type to fp16 for FP16 mode
!dssd export --gpu_index=$GPU_INDEX \
                 -m $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/dssd_resnet18_epoch_$EPOCH.tlt \
                 -k $KEY \
                 -o $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt \
                 -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
                 --batch_size 16 \
                 --data_type fp32 \
                 --gen_ds_config

# Uncomment to export in INT8 mode (generate calibration cache file).
# !dssd export --gpu_index=$GPU_INDEX \
#                  -m $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/dssd_resnet18_epoch_$EPOCH.tlt  \
#                  -o $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt \
#                  -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
#                  -k $KEY \
#                  --cal_image_dir $LOCAL_DATA_DIR/testing/image_2 \
#                  --data_type int8 \
#                  --batch_size 16 \
#                  --batches 10 \
#                  --cal_cache_file $LOCAL_EXPERIMENT_DIR/export/cal.bin  \
#                  --cal_data_file $LOCAL_EXPERIMENT_DIR/export/cal.tensorfile \
#                  --gen_ds_config

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-13 17:20:18,533 [INFO] root: Building exporter object.
2022-09-13 17:20:49,359 [INFO] root: Exporting the model.
2022-09-13 17:20:49,360 [INFO] root: Using input nodes: ['Input']
2022-09-13 17:20:49,360 [INFO] root: Using output nodes: ['NMS']
2022-09-13 17:20:49,360 [INFO] iva.common.export.keras_exporter: Using input nodes: ['Input']
2022-09-13 17:20:49,360 [INFO] iva.common.export.keras_exporter: Using output nodes: ['NMS']
2022-09-13 17:20:49,360 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_retrain_resnet18_kitti.txt
NOTE: UFF has been tested with TensorFlow 1.14.0.
Converting NMS as custom op: NMS_TRT
Converting FirstDimTile_5 as custom op: BatchTilePlugin_TRT
Converting FirstDimTile_4 as custom op: BatchTilePlugin_TRT
Converting FirstDimTile_3 as custom op: BatchTilePlugin_TRT
Converting FirstDimTile_2 as custom op: BatchTilePlugin_TRT
Converting FirstDimTile_1 as custom op: 

`Note:` In this example, for ease of execution we restrict the number of calibrating batches to 10. TAO Toolkit recommends the use of at least 10% of the training dataset for int8 calibration.

If you train a QAT model, you may only export in INT8 mode using following code block. This generates an etlt file and the corresponding calibration cache. You can throw away the calibration cache and just use the etlt file in tao-converter or DeepStream for FP32 or FP16 mode. But please note this gives sub-optimal results. If you want to deploy in FP32 or FP16, you should disable QAT in training.

In [ ]:
# Uncomment to export QAT model in INT8 mode (generate calibration cache file).
# !rm -rf $LOCAL_EXPERIMENT_DIR/export
# !mkdir -p $LOCAL_EXPERIMENT_DIR/export
# !dssd export --gpu_index=$GPU_INDEX \
#                  -m $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights/dssd_resnet18_epoch_$EPOCH.tlt  \
#                  -o $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt \
#                  -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
#                  -k $KEY \
#                  --data_type int8 \
#                  --cal_cache_file $LOCAL_EXPERIMENT_DIR/export/cal.bin \
#                  --gen_ds_config

In [11]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export

Exported model:
------------
total 27M
-rw-r--r-- 1 jupyter jupyter 27M Sep 13 17:32 dssd_resnet18_epoch_020.etlt
-rw-r--r-- 1 jupyter jupyter  34 Sep 13 17:32 labels.txt
-rw-r--r-- 1 jupyter jupyter 223 Sep 13 17:32 nvinfer_config.txt


Verify engine generation using the `tao-converter` utility included with the docker.

The `tao-converter` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please instantiate this docker and execute the `tao-converter` command, with the exported `.etlt` file and calibration cache (for int8 mode) on your target device. The tao-converter utility included in this docker only works for x86 devices, with discrete NVIDIA GPU's. 

For the jetson devices, please download the tao-converter for jetson from the dev zone link [here](https://developer.nvidia.com/tao-converter). 

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [12]:
# Convert to TensorRT engine (FP32)
!converter -k $KEY \
                   -d 3,300,300 \
                   -o NMS \
                   -e $LOCAL_EXPERIMENT_DIR/export/trt.engine \
                   -m 16 \
                   -t fp32 \
                   -i nchw \
                   $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt

# Convert to TensorRT engine (FP16)
# !converter -k $KEY \
#                    -d 3,300,300 \
#                    -o NMS \
#                    -e $LOCAL_EXPERIMENT_DIR/export/trt.engine \
#                    -m 16 \
#                    -t fp16 \
#                    -i nchw \
#                    $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt

# Convert to TensorRT engine (INT8).
# !converter -k $KEY  \
#                    -d 3,300,300 \
#                    -o NMS \
#                    -c $LOCAL_EXPERIMENT_DIR/export/cal.bin \
#                    -e $LOCAL_EXPERIMENT_DIR/export/trt.engine \
#                    -b 8 \
#                    -m 16 \
#                    -t int8 \
#                    -i nchw \
#                    $LOCAL_EXPERIMENT_DIR/export/dssd_resnet18_epoch_$EPOCH.etlt

[INFO] [MemUsageChange] Init CUDA: CPU +252, GPU +0, now: CPU 258, GPU 487 (MiB)
[INFO] [MemUsageSnapshot] Builder begin: CPU 312 MiB, GPU 487 MiB
[INFO] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +351, GPU +160, now: CPU 686, GPU 647 (MiB)
[INFO] [MemUsageChange] Init cuDNN: CPU +273, GPU +132, now: CPU 959, GPU 779 (MiB)
[WARNING] Detected invalid timing cache, setup a local cache instead
[INFO] Some tactics do not have sufficient workspace memory to run. Increasing workspace size may increase performance, please check verbose output.
[INFO] Detected 1 inputs and 2 output network tensors.
[INFO] Total Host Persistent Memory: 116688
[INFO] Total Device Persistent Memory: 58925056
[INFO] Total Scratch Memory: 21096704
[INFO] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 24 MiB, GPU 64 MiB
[INFO] [MemUsageChange] Init cuBLAS/cuBLASLt: CPU +0, GPU +8, now: CPU 1299, GPU 983 (MiB)
[INFO] [MemUsageChange] Init cuDNN: CPU +0, GPU +8, now: CPU 1299, GPU 991 (MiB)
[I

In [13]:
print('Exported engine:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/trt.engine

Exported engine:
------------
-rw-r--r-- 1 jupyter jupyter 61M Sep 13 17:34 /home/jupyter/dssd/export/trt.engine


## 11. Verify the deployed model <a class="anchor" id="head-11"></a>
Verify the converted engine by visualizing TensorRT inferences.

In [16]:
# Infer using TensorRT engine

# The engine batch size once created, cannot be alterred. So if you wish to run with a different batch-size,
# please re-run tlt-convert.

!dssd inference --gpu_index=$GPU_INDEX \
                    -m $LOCAL_EXPERIMENT_DIR/export/trt.engine \
                    -e $LOCAL_SPECS_DIR/dssd_retrain_resnet18_kitti.txt \
                    -i $LOCAL_DATA_DIR/test_samples \
                    -o $LOCAL_EXPERIMENT_DIR/dssd_infer_images \
                    -t 0.4

Using TensorFlow backend.
Using TensorFlow backend.
2022-09-13 17:37:52,190 [INFO] iva.ssd.utils.spec_loader: Merging specification from /home/jupyter/dssd/specs/dssd_retrain_resnet18_kitti.txt

2022-09-13 17:37:52,193 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


2022-09-13 17:37:52,193 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.


2022-09-13 17:37:52,196 [WARNING] tensorflow: From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

Using TensorRT engine for inference, setting batch size to engine's one: 16
100%|██████████████████████

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'dssd_infer_images' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)